<div style="text-align:left;">
  <a href="https://code213.tech/" target="_blank">
    <img src="code213.PNG" alt="code213">
  </a>
  <p><em>prepared by Latreche Sara</em></p>
</div>

# 10.7 - Model Implementations in PyTorch

PyTorch provides flexible ways to **define and implement models** for deep learning.  

### Key Points
1. **Sequential Models**
   - Use `nn.Sequential` to stack layers in order  
   - Easy for simple feedforward architectures

2. **Custom Models**
   - Subclass `nn.Module`  
   - Define `__init__()` for layers and `forward()` for forward pass  
   - Allows complete flexibility for any architecture

3. **Predefined Models**
   - PyTorch provides models in `torchvision.models`  
   - Useful for transfer learning



In this notebook, we will cover:  
- Implementing a simple feedforward network  
- Defining custom models with `nn.Module`  
- Using models with GPU and DataLoader


## Table of Contents

- [1 - Sequential Model](#1)
- [2 - Custom Model with nn.Module](#2)
- [3 - Using Predefined Models](#3)
- [4 - Training a Model](#4)
- [5 - Practice Exercises](#5)


<a name='1'></a>
## 1 - Sequential Model

`nn.Sequential` allows you to **stack layers sequentially** to define simple feedforward networks.

### Key Points
- Layers are executed in the order they are added  
- Easy to implement simple networks without defining a `forward()` method  
- Supports any layer type: `Linear`, `ReLU`, `Dropout`, etc.

Example architecture:
- Input: 4 features  
- Hidden layer: 5 neurons, ReLU activation  
- Output: 3 neurons


In [ ]:
import torch
import torch.nn as nn

# Define a simple sequential model
model = nn.Sequential(
    nn.Linear(4, 5),
    nn.ReLU(),
    nn.Linear(5, 3)
)

# Sample input
x = torch.randn(2, 4)

# Forward pass
output = model(x)
print("Sequential model output:\n", output)


<a name='2'></a>
## 2 - Custom Model with nn.Module

For more complex architectures, we can **subclass `nn.Module`**.  

### Key Points
- Define layers in `__init__()`  
- Define forward computation in `forward()`  
- Provides full flexibility for branching, multiple inputs, skip connections, etc.

Example architecture:
- Input: 4 features  
- Hidden layer 1: 6 neurons, ReLU  
- Hidden layer 2: 3 neurons, ReLU  
- Output: 2 neurons


In [1]:
import torch
import torch.nn as nn

# Define custom model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(4, 6)
        self.fc2 = nn.Linear(6, 3)
        self.fc3 = nn.Linear(3, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create model instance
model = CustomModel()

# Sample input
x = torch.randn(2, 4)

# Forward pass
output = model(x)
print("Custom model output:\n", output)


Custom model output:
 tensor([[1.0381, 0.3685],
        [0.8170, 0.1110]], grad_fn=<AddmmBackward0>)


<a name='3'></a>
## 3 - Using Predefined Models

PyTorch provides **pretrained models** in `torchvision.models` for tasks like image classification, segmentation, and more.  

### Key Points
- Pretrained models can be used for **transfer learning**  
- Common models: `resnet18`, `vgg16`, `alexnet`  
- You can modify the **final layers** for your own number of classes  

Example workflow:
1. Load a pretrained model  
2. Replace the final layer(s) if needed  
3. Move model to GPU if available  
4. Use DataLoader to feed data


In [2]:
import torch
import torchvision.models as models

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pretrained ResNet18
resnet18 = models.resnet18(pretrained=True)

# Modify the final layer for 10 classes
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, 10)

# Move model to device
resnet18 = resnet18.to(device)

# Sample input: batch size 2, 3 channels, 224x224 images
x = torch.randn(2, 3, 224, 224).to(device)

# Forward pass
output = resnet18(x)
print("Pretrained ResNet18 output shape:", output.shape)


c:\Users\MatenTech\AppData\Local\anaconda3\envs\bd\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MatenTech\AppData\Local\anaconda3\envs\bd\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\MatenTech/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


Pretrained ResNet18 output shape: torch.Size([2, 10])


<a name='4'></a>
## 4 - Training a Model

Once a model is defined, we can train it using PyTorch's **training loop**.  

### Key Points
- Move both **model and data** to the same device (CPU or GPU)  
- Define **loss function** and **optimizer**  
- Typical training loop:
  1. Forward pass: compute predictions  
  2. Compute loss  
  3. Backward pass: compute gradients  
  4. Optimizer step: update weights  
  5. Repeat for multiple epochs


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Sample dataset
x = torch.randn(20, 4)
y = torch.randint(0, 2, (20,))
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)

# Simple model
model = nn.Linear(4, 2).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(3):
    for x_batch, y_batch in dataloader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        # Forward pass
        outputs = model(x_batch)
        loss = loss_fn(outputs, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.6435
Epoch 2, Loss: 0.6590
Epoch 3, Loss: 0.6754


<a name='5'></a>
## 5 - Practice Exercises

Try the following exercises to reinforce your understanding of **PyTorch model implementations**:

---

### **Exercise 1: Sequential Model**
- Create a `nn.Sequential` model with:
  - Input: 3 features
  - Hidden layer: 4 neurons, ReLU activation
  - Output: 2 neurons
- Forward a random input tensor and print the output

---

### **Exercise 2: Custom Model**
- Subclass `nn.Module` to create a model with:
  - Input: 3 features
  - Hidden layer 1: 5 neurons, ReLU
  - Hidden layer 2: 3 neurons, ReLU
  - Output: 2 neurons
- Forward a random input tensor and print the output

---

### **Exercise 3: Pretrained Model**
- Load a pretrained `resnet18`  
- Modify the final layer to have 5 output classes  
- Forward a random tensor of shape `(2, 3, 224, 224)` and print the output shape

---

### **Exercise 4: Training Loop**
- Create a small dataset with 12 samples, 3 features each, and labels 0 or 1  
- Define a simple linear model  
- Train it for 2 epochs using `CrossEntropyLoss` and SGD optimizer  
- Print the loss after each batch


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ----------------------------
# Exercise 1: Sequential Model
# ----------------------------
seq_model = nn.Sequential(
    nn.Linear(3, 4),
    nn.ReLU(),
    nn.Linear(4, 2)
).to(device)

x = torch.randn(1, 3).to(device)
print("Sequential model output:", seq_model(x))

# ----------------------------
# Exercise 2: Custom Model
# ----------------------------
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(3, 5)
        self.fc2 = nn.Linear(5, 3)
        self.fc3 = nn.Linear(3, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

custom_model = CustomModel().to(device)
x = torch.randn(1, 3).to(device)
print("Custom model output:", custom_model(x))

# ----------------------------
# Exercise 3: Pretrained Model
# ----------------------------
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 5)
resnet18 = resnet18.to(device)
x_img = torch.randn(2, 3, 224, 224).to(device)
print("Pretrained ResNet18 output shape:", resnet18(x_img).shape)

# ----------------------------
# Exercise 4: Training Loop
# ----------------------------
data = torch.randn(12, 3)
labels = torch.randint(0, 2, (12,))
dataset = TensorDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

model = nn.Linear(3, 2).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(2):
    for x_batch, y_batch in dataloader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        loss = loss_fn(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Sequential model output: tensor([[0.4797, 0.6344]], grad_fn=<AddmmBackward0>)
Custom model output: tensor([[0.4419, 0.2808]], grad_fn=<AddmmBackward0>)
Pretrained ResNet18 output shape: torch.Size([2, 5])
Epoch 1, Loss: 1.2573
Epoch 1, Loss: 0.7442
Epoch 1, Loss: 0.8245
Epoch 2, Loss: 1.6021
Epoch 2, Loss: 0.7905
Epoch 2, Loss: 0.3885
